# 5. Ensembles de Arboles de Decision

## 5.4 GBDT LightGBM

La técnica de Gradient Boosting fue creada por Jerome H. Friedman en 1999 - 2001
<br>Se implementaron librerías ineficientes
<br>En 2016 se crea XGBoost, en 2017 LightGBM

El Gradient Boosting of Decision Trees es un ensemble de árboles de decisión, para un nuevo registro la predicción se hace sumando el score que cada arbol asigna a ese registro.

En GBDT la construccion de los árboles es secuencial, ya que el arbol n-simo se genera para predecir el error del modelo conformado por los  n-1 arboles previos, aunque sea un arbol de clasificación lo que se predice es un numero real mediante un arbol de regresión.


<br>Qué tipo de perturbaciones se realiza LightGBM

*   Se perturba el dataset, seleccionando para cada arbol un subconjunto de las columnas.
*   El algortimo de arbol de decisión no presenta perturbaciones

Cada arbolito de LightGBM se entrena sobre un dataset perturbado, que en principio posee :
* todos los registros del dataset original
* solo un porcentaje *feature_fraction* de las columnas originales del dataset

#### 5.4.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 5.4.2  LightGBM, una corrida

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,649929,34.8,1439382,76.9,1359064,72.6
Vcells,1209847,9.3,8388608,64.0,1924969,14.7


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

Loading required package: data.table

Loading required package: rpart

Loading required package: rlist

Loading required package: lightgbm



Aqui debe cargar SU semilla primigenia

In [4]:
PARAM <- list()
PARAM$experimento <- 5421
PARAM$semilla_primigenia <- 314159

# estos hiperparametros de LightGBM surgieron de una Bayesian Optimization
PARAM$lgb$num_iterations <- 10000  # cantidad de arbolitos
PARAM$lgb$learning_rate <- 0.027
PARAM$lgb$feature_fraction <- 0.8
PARAM$lgb$min_data_in_leaf <- 76
PARAM$lgb$num_leaves <- 2
PARAM$lgb$max_bin <- 31


In [5]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [6]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv", stringsAsFactors = TRUE)

In [7]:
# paso la clase a binaria que tome valores {0,1}  enteros
# set trabaja con la clase  POS = { BAJA+1, BAJA+2 }
# esta estrategia es MUY importante
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2", "BAJA+1"), 1L, 0L)]

In [8]:
# los campos que se van a utilizar
campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01"))

In [9]:
# establezco donde entreno
dataset[, train := 0L]
dataset[foto_mes %in% c(202107), train := 1L]

In [10]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[train == 1L, campos_buenos, with = FALSE]),
  label= dataset[train == 1L, clase01]
)

In [11]:
# genero el modelo
# estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
set.seed( PARAM$semilla_primigenia ) # Establezco la semilla aleatoria

modelo <- lgb.train(
  data= dtrain,
  param= list(
    objective= "binary",
    max_bin= PARAM$lgb$max_bin,
    learning_rate= PARAM$lgb$learning_rate,
    num_iterations= PARAM$lgb$num_iterations,
    num_leaves= PARAM$lgb$num_leaves,
    min_data_in_leaf= PARAM$lgb$min_data_in_leaf,
    feature_fraction= PARAM$lgb$feature_fraction,
    seed= PARAM$semilla_primigenia
  )
)


[LightGBM] [Info] Number of positive: 2385, number of negative: 162211
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025498 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3612
[LightGBM] [Info] Number of data points in the train set: 164596, number of used features: 153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014490 -> initscore=-4.219699
[LightGBM] [Info] Start training from score -4.219699


In [12]:
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelo))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file = archivo_importancia,
  sep = "\t"
)


In [13]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo, "modelo.txt" )

In [14]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)


In [15]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file = "prediccion.txt",
  sep = "\t"
)

In [16]:
# subidas a Kaggle
# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

In [17]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

cortes <- seq(9000, 13500, by = 500)

for (envios in cortes) {

  tb_prediccion[, Predicted := 0L]
  tb_prediccion[1:envios, Predicted := 1L]

  archivo_kaggle <- paste0("KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # subida a Kaggle
  comando <- "kaggle competitions submit"
  competencia <- "-c labo-i-2025-rosario"
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'num_iterations=", PARAM$lgb$num_iterations,
    "  learning_rate=", PARAM$lgb$learning_rate,
    "  feature_fraction=", PARAM$lgb$feature_fraction,
    "  min_data_in_leaf=", PARAM$lgb$min_data_in_leaf,
    "  num_leaves=",PARAM$lgb$num_leaves,
    "  max_bin=", PARAM$lgb$max_bin,
  "'" )

  linea <- paste( comando, competencia, arch, mensaje)
  salida <- system(linea, intern=TRUE)
  cat(salida)
}

Successfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 RosarioSuccessfully submitted to LaboI 2025 Rosario



---



### 5.4.3  LightGBM  optimizacion de hiperparámetros

La optimizacion de los hiperparámetros de LightGBM mediante el método de optimizacion bayesiana será su *caballito de batalla* durante la asignatura !

limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,650030,34.8,1439394,76.9,1154840,61.7
Vcells,1210124,9.3,8388608,64.0,1924969,14.7


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes

Loading required package: rlist

Loading required package: lightgbm

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




Aqui debe cargar SU semilla primigenia

In [3]:
PARAM <- list()
PARAM$experimento <- 5431
PARAM$semilla_primigenia <- 314159

# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos
PARAM$trainingstrategy$undersampling <- 1.0

PARAM$hyperparametertuning$iteraciones <- 500
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 117000
PARAM$hyperparametertuning$NEG_ganancia <- -3000

# Aqui se cargan los bordes de los hiperparametros
PARAM$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.01, upper = 0.3),
  makeIntegerParam("num_leaves", lower = 8L, upper = 1024L),
  makeNumericParam("feature_fraction", lower = 0.1, upper = 1.0),
  makeIntegerParam("min_data_in_leaf", lower = 1L, upper = 8000L),
  makeIntegerParam("envios", lower = 5000L, upper = 15000L),
  makeNumericParam("bagging_fraction", lower = 0.1, upper = 1.0), #Agregado
  makeNumericParam("lambda_l1", lower = 0, upper = 2.0) #Agregado
)

In [4]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch = NA, folder = "./work/",
    ext = ".txt", verbose = TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse = "\t"), "\n"
      )

      cat(linea, file = archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file = archivo, append = TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [5]:
# esta funcion calcula internamente la ganancia de la prediccion probs
# es llamada por lightgbm luego de construir cada  arbolito

fganancia_logistic_lightgbm <- function(probs, datos) {
  vpesos <- get_field(datos, "weight")

  # vector de ganancias
  vgan <- ifelse(vpesos == 1.0000002, PARAM$hyperparametertuning$POS_ganancia,
    ifelse(vpesos == 1.0000001, PARAM$hyperparametertuning$NEG_ganancia,
      PARAM$hyperparametertuning$NEG_ganancia /
        PARAM$trainingstrategy$undersampling
    )
  )

  tbl <- as.data.table(list("vprobs" = probs, "vgan" = vgan))
  setorder(tbl, -vprobs)
  ganancia <- tbl[1:GLOBAL_envios, sum(vgan)]

  return(list(
    "name" = "ganancia",
    "value" = ganancia,
    "higher_better" = TRUE
  ))
}


In [6]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales,
# la semilla del mal ...


EstimarGanancia_lightgbm <- function(x) {
  gc() # libero memoria

  # llevo el registro de la iteracion por la que voy
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # para usar en fganancia_logistic_lightgbm
  # asigno la variable global
  GLOBAL_envios <<- as.integer(x$envios / PARAM$hyperparametertuning$xval_folds)

  # cantidad de folds para cross validation
  kfolds <- PARAM$hyperparametertuning$xval_folds

  param_basicos <- list(
    objective= "binary",
    metric= "custom",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    max_bin= 31, # por ahora, lo dejo fijo
    num_iterations= 9999, # valor grande, lo limita early_stopping_rounds
    force_row_wise= TRUE, # para evitar warning
    seed= ksemilla_azar1
  )

  # el parametro discolo, que depende de otro
  param_variable <- list(
    early_stopping_rounds =
      as.integer(50 + 5 / x$learning_rate)
  )

  param_completo <- c(param_basicos, param_variable, x)

  set.seed(ksemilla_azar1)
  modelocv <- lgb.cv(
    data= dtrain,
    eval= fganancia_logistic_lightgbm,
    stratified= TRUE, # sobre el cross validation
    nfold= kfolds, # folds del cross validation
    param= param_completo,
    verbose= -100
  )

  # obtengo la ganancia
  ganancia <- unlist(modelocv$record_evals$valid$ganancia$eval)[modelocv$best_iter]

  ganancia_normalizada <- ganancia * kfolds # normailizo la ganancia

  # asigno el mejor num_iterations
  param_completo$num_iterations <- modelocv$best_iter
  # elimino de la lista el componente
  param_completo["early_stopping_rounds"] <- NULL


  # el lenguaje R permite asignarle ATRIBUTOS a cualquier variable
  # esta es la forma de devolver un parametro extra
  attr(ganancia_normalizada, "extras") <-
    list("num_iterations" = modelocv$best_iter)

  # logueo
  xx <- param_completo
  xx$ganancia <- ganancia_normalizada # le agrego la ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = klog)

  # Voy registrando la importancia de variables
  if (ganancia_normalizada > GLOBAL_gananciamax) {
    GLOBAL_gananciamax <<- ganancia_normalizada
    modelo <- lgb.train(
      data = dtrain,
      param = param_completo,
      verbose = -100
    )

    tb_importancia <- as.data.table(lgb.importance(modelo))
    archivo_importancia <- paste0("impo_", GLOBAL_iteracion, ".txt")
    fwrite(tb_importancia,
      file = archivo_importancia,
      sep = "\t" )

    loguear(xx, arch = klog_mejor)
  }

  return(ganancia_normalizada)
}


aqui se inicia el programa

In [7]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [8]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )
ksemilla_azar1 <- PARAM$semillas[1]
ksemilla_azar2 <- PARAM$semillas[2]


In [9]:
# en estos archivos quedan los resultados

kbayesiana <- paste0(PARAM$experimento, ".RDATA")
klog <- paste0(PARAM$experimento, ".txt")
klog_mejor <- paste0(PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_gananciamax <- -1 # inicializo la variable global

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
  GLOBAL_gananciamax <- tabla_log[, max(ganancia)]
}


In [10]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [11]:
dataset <- dataset[foto_mes %in% c(202107)]

In [12]:
# paso la clase a binaria que tome valores {0,1}  enteros
dataset[
  foto_mes %in% c(202107),
  clase01 := ifelse(clase_ternaria == "CONTINUA", 0L, 1L)
]

In [13]:
# los campos que se van a utilizar
campos_buenos <- setdiff(
  colnames(dataset),
  c("clase_ternaria", "clase01", "azar", "training")
)


In [14]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(ksemilla_azar2)
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]
dataset[
  foto_mes %in% c(202107) &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [15]:
# dejo los datos en el formato que necesita LightGBM
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label = dataset[training == 1L, clase01],
  weight = dataset[training == 1L, ifelse(clase_ternaria == "BAJA+2", 1.0000002, ifelse(clase_ternaria == "BAJA+1", 1.0000001, 1.0))],
  free_raw_data = FALSE
)


In [16]:
# Aqui comienza la configuracion de la Bayesian Optimization
funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar, # la funcion que voy a maximizar
  minimize = FALSE, # estoy Maximizando la ganancia
  noisy = TRUE,
  par.set = PARAM$hs, # definido al comienzo del programa
  has.simple.signature = FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600, # se graba cada 600 segundos
  save.file.path = kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters = PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type = "se",
  covtype = "matern3_2",
  control = list(trace = TRUE)
)


In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else {
  run <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.



20250605 212524	binary	custom	TRUE	TRUE	FALSE	-100	31	111	TRUE	531673	0.162751880437468	789	0.795007657180707	5015	9810	0.687453841966011	1.45448340738325	59130000	1
20250605 212530	binary	custom	TRUE	TRUE	FALSE	-100	31	111	TRUE	531673	0.162751880437468	789	0.795007657180707	5015	9810	0.687453841966011	1.45448340738325	59130000	1
20250605 212559	binary	custom	TRUE	TRUE	FALSE	-100	31	151	TRUE	531673	0.168665953657889	561	0.318468382091461	4857	9318	0.943020798709144	1.04712087449817	59055000	2
20250605 212623	binary	custom	TRUE	TRUE	FALSE	-100	31	118	TRUE	531673	0.212989273317009	642	0.411942562959822	6417	12850	0.40437429361378	0.600754726278995	58530000	3
20250605 212711	binary	custom	TRUE	TRUE	FALSE	-100	31	283	TRUE	531673	0.142678953167633	457	0.450505346165404	3576	11248	0.367609061638359	1.31838257673995	59745000	4
20250605 212721	binary	custom	TRUE	TRUE	FALSE	-100	31	283	TRUE	531673	0.142678953167633	457	0.450505346165404	3576	11248	0.367609061638359	1.31838257673995	59745000	4
2

[mbo] 0: learning_rate=0.163; num_leaves=789; feature_fraction=0.795; min_data_in_leaf=5015; envios=9810; bagging_fraction=0.687; lambda_l1=1.45 : y = 5.91e+07 : 29.0 secs : initdesign

[mbo] 0: learning_rate=0.169; num_leaves=561; feature_fraction=0.318; min_data_in_leaf=4857; envios=9318; bagging_fraction=0.943; lambda_l1=1.05 : y = 5.91e+07 : 28.1 secs : initdesign

[mbo] 0: learning_rate=0.213; num_leaves=642; feature_fraction=0.412; min_data_in_leaf=6417; envios=12850; bagging_fraction=0.404; lambda_l1=0.601 : y = 5.85e+07 : 24.2 secs : initdesign

[mbo] 0: learning_rate=0.143; num_leaves=457; feature_fraction=0.451; min_data_in_leaf=3576; envios=11248; bagging_fraction=0.368; lambda_l1=1.32 : y = 5.97e+07 : 59.0 secs : initdesign

[mbo] 0: learning_rate=0.0519; num_leaves=218; feature_fraction=0.374; min_data_in_leaf=2526; envios=12318; bagging_fraction=0.556; lambda_l1=0.841 : y = 5.91e+07 : 33.3 secs : initdesign

[mbo] 0: learning_rate=0.127; num_leaves=547; feature_fraction=0

20250605 214748	binary	custom	TRUE	TRUE	FALSE	-100	31	534	TRUE	531673	0.0104476244117705	905	0.388980039034146	3	14508	0.256006530527718	1.62680338501962	59085000	29


[mbo] 1: learning_rate=0.0104; num_leaves=905; feature_fraction=0.389; min_data_in_leaf=3; envios=14508; bagging_fraction=0.256; lambda_l1=1.63 : y = 5.91e+07 : 272.3 secs : infill_ei



...

[mbo] 145: learning_rate=0.0103; num_leaves=975; feature_fraction=0.148; min_data_in_leaf=184; envios=11748; bagging_fraction=0.329; lambda_l1=0.641 : y = 6.05e+07 : 158.7 secs : infill_ei



20250606 025313	binary	custom	TRUE	TRUE	FALSE	-100	31	619	TRUE	531673	0.0109017615703278	871	0.749735423874772	1669	7861	0.999537458437351	1.99557445973013	59580000	174


[mbo] 146: learning_rate=0.0109; num_leaves=871; feature_fraction=0.75; min_data_in_leaf=1669; envios=7861; bagging_fraction=1; lambda_l1=2 : y = 5.96e+07 : 119.7 secs : infill_ei



20250606 025615	binary	custom	TRUE	TRUE	FALSE	-100	31	1149	TRUE	531673	0.0136170257998371	922	0.946485307251403	1407	12076	0.159501358184979	1.22328802536345	61215000	175


[mbo] 147: learning_rate=0.0136; num_leaves=922; feature_fraction=0.946; min_data_in_leaf=1407; envios=12076; bagging_fraction=0.16; lambda_l1=1.22 : y = 6.12e+07 : 180.6 secs : infill_ei





---

